## Task 1 - SQL

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null



!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz


!tar xf spark-3.0.1-bin-hadoop3.2.tgz


!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

### Build SparkSession:

In [2]:
spark = SparkSession.builder.appName('spark_HW2').getOrCreate()

### Read the json file:

In [3]:
from google.colab import files
uploaded = files.upload()

Saving DataFrames_sample.json to DataFrames_sample.json
Saving ITI_data.csv to ITI_data.csv
Saving test1.csv to test1.csv
Saving test3.csv to test3.csv


In [6]:
import io
DataFrames_json = spark.read.json('DataFrames_sample.json')

### Display the schema:


In [7]:
DataFrames_json.printSchema()

root
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- HDD: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- W: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Year: long (nullable = true)



### Get all the data when "Model" equal "MacBook Pro":




In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [11]:
DataFrames_json.where(col('Model')=='MacBook Pro').show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Create TempView:

In [14]:
DataFrames_json.createOrReplaceTempView('DataFrames_json_view')

### Display "RAM"column and count "RAM" column:

In [15]:
spark.sql('SELECT RAM,COUNT(RAM) AS RAM_Count FROM DataFrames_json_view GROUP BY RAM').show()

+----+---------+
| RAM|RAM_Count|
+----+---------+
|64GB|        1|
|16GB|        1|
| 8GB|        2|
+----+---------+



### Get all columns when "Year" column equal "2015"  

In [22]:
spark.sql("SELECT * FROM DataFrames_json_view WHERE Year=='2015'").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all when "Model" start with "M":

In [26]:
spark.sql("SELECT * FROM DataFrames_json_view WHERE Model LIKE 'M%'").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data when "Model" column equal "MacBook Pro"

In [27]:
spark.sql("SELECT * FROM DataFrames_json_view WHERE Model == 'MacBook Pro'").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [31]:
spark.sql("SELECT * FROM DataFrames_json_view WHERE Model=='Macbook' OR RAM=='8GB'").show()

+----+----+---------+---+-----------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|    MacBook|8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air|8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+---+----------+-----+------+----+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [32]:
spark.sql("SELECT * FROM DataFrames_json_view WHERE Model=='iMac' OR D>=8").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
| 8.0|20.3|  1TB SSD|  4|       iMac|64GB|       27"| 25.6|  20.8|2017|
+----+----+---------+---+-----------+----+----------+-----+------+----+



## Task 2


### Read "test1" dataset:

In [36]:
Test1_csv = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [44]:
Test1_csv.show(truncate=False)

+---------+---+----------+------+
|Name     |age|Experience|Salary|
+---------+---+----------+------+
|Krish    |31 |10        |30000 |
|Sudhanshu|30 |8         |25000 |
|Sunny    |29 |4         |20000 |
|Paul     |24 |3         |20000 |
|Harsha   |21 |1         |15000 |
|Shubham  |23 |2         |18000 |
+---------+---+----------+------+



### Display Salary of the people less than or equal to 20000

In [43]:
Test1_csv.where(col('Salary')<=20000).show(truncate=False)

+-------+---+----------+------+
|Name   |age|Experience|Salary|
+-------+---+----------+------+
|Sunny  |29 |4         |20000 |
|Paul   |24 |3         |20000 |
|Harsha |21 |1         |15000 |
|Shubham|23 |2         |18000 |
+-------+---+----------+------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [46]:
Test1_csv.where(col('Salary') <= 20000).where(col('Salary') >= 15000).show(truncate=False)

+-------+---+----------+------+
|Name   |age|Experience|Salary|
+-------+---+----------+------+
|Sunny  |29 |4         |20000 |
|Paul   |24 |3         |20000 |
|Harsha |21 |1         |15000 |
|Shubham|23 |2         |18000 |
+-------+---+----------+------+



## Task 3 

### Read "test3" dataset:

In [53]:
Test3_csv = spark.read.csv('test3.csv', inferSchema=True, header=True)

### Display dataset

In [54]:
Test3_csv.show(truncate=False)

+---------+------------+------+
|Name     |Departments |salary|
+---------+------------+------+
|Krish    |Data Science|10000 |
|Krish    |IOT         |5000  |
|Mahesh   |Big Data    |4000  |
|Krish    |Big Data    |4000  |
|Mahesh   |Data Science|3000  |
|Sudhanshu|Data Science|20000 |
|Sudhanshu|IOT         |10000 |
|Sudhanshu|Big Data    |5000  |
|Sunny    |Data Science|10000 |
|Sunny    |Big Data    |2000  |
+---------+------------+------+



### Display schema

In [49]:
Test3_csv.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



In [62]:
type(Test3_csv)

pyspark.sql.dataframe.DataFrame

### Group by "Name" column and using sum function on "Name" column

In [94]:
test3_df = Test3_csv.toPandas()
test3_df

,Name,Departments,salary
0,Krish,Data Science,10000
1,Krish,IOT,5000
2,Mahesh,Big Data,4000
3,Krish,Big Data,4000
4,Mahesh,Data Science,3000
5,Sudhanshu,Data Science,20000
6,Sudhanshu,IOT,10000
7,Sudhanshu,Big Data,5000
8,Sunny,Data Science,10000
9,Sunny,Big Data,2000


In [66]:
test3_df.groupby('Name').sum()

,salary
Name,
Krish,19000
Mahesh,7000
Sudhanshu,35000
Sunny,12000


### Group by "Name" column and using avg function on "Name" column

In [72]:
test3_df.groupby('Name').mean()

,salary
Name,
Krish,6333.333333
Mahesh,3500.000000
Sudhanshu,11666.666667
Sunny,6000.000000


### Group by "Departments" column and using sum function on "Departments" column

In [74]:
test3_df.groupby('Departments').sum()

,salary
Departments,
Big Data,15000
Data Science,43000
IOT,15000


### Group by "Departments" column and using mean function on "Departments" column:




In [77]:
test3_df.groupby('Departments').mean()

,salary
Departments,
Big Data,3750
Data Science,10750
IOT,7500


### Group by "Departments" column and using count function on "Departments" column:



In [89]:
Test3_csv.groupby('Departments').agg(count('Departments')).show()

+------------+------------------+
| Departments|count(Departments)|
+------------+------------------+
|         IOT|                 2|
|    Big Data|                 4|
|Data Science|                 4|
+------------+------------------+



In [79]:
test3_df.groupby('Departments').count()

,Name,salary
Departments,,
Big Data,4,4
Data Science,4,4
IOT,2,2


### Apply agg to using sum function get the total of salaries

In [84]:
Test3_csv.agg(sum('salary').alias('Sum Salaries')).show()

+------------+
|Sum Salaries|
+------------+
|       73000|
+------------+



## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [95]:
ITI_data = spark.read.csv('ITI_data.csv', inferSchema=True, header=True)

In [96]:
ITI_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [108]:
ITI_data.dtypes

[('Ship_name', 'string'),
 ('Cruise_line', 'string'),
 ('Age', 'int'),
 ('Tonnage', 'double'),
 ('passengers', 'double'),
 ('length', 'double'),
 ('cabins', 'double'),
 ('passenger_density', 'double'),
 ('crew', 'double')]

In [97]:
trainDF, testDF = ITI_data.randomSplit([.8,.2],seed=42)
print(f"There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set")

There are 133 rows in the training set, and 25 in the test set


In [109]:
categoricalCols = [field for(field, datatype) in trainDF.dtypes if datatype=='string']
categoricalCols

['Ship_name', 'Cruise_line']

In [110]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Ship_name_Index', 'Cruise_line_Index']

In [112]:
oheOutputCols = [x + "_ohe" for x in categoricalCols]
oheOutputCols

['Ship_name_ohe', 'Cruise_line_ohe']

In [113]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid='skip')

### OneHotEncoder 


In [115]:
ohEncoder = OneHotEncoder(inputCols=indexOutputCols, outputCols=oheOutputCols)
ohEncoder

OneHotEncoder_778133188cee

In [118]:
numericCols = [field for (field,dataType) in trainDF.dtypes
              if (((dataType=='double') or (dataType=='int')) and (field !='crew') )]
numericCols

['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']

In [119]:
assemblerInputs = oheOutputCols + numericCols
assemblerInputs

['Ship_name_ohe',
 'Cruise_line_ohe',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density']

###Use VectorAssembler to merge all columns into one column:

In [120]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol='features')

### Create a Regression Model 

In [148]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(labelCol='crew',featuresCol='features')

### Creating a Pipeline

In [149]:
from pyspark.ml import Pipeline

pipeline =Pipeline(stages = [stringIndexer, ohEncoder, vecAssembler, rf])

In [150]:
pipelineModel = pipeline.fit(trainDF)

In [151]:
predDF = pipelineModel.transform(testDF)

In [152]:
predDF.select('crew', 'prediction').show(5)

+----+------------------+
|crew|        prediction|
+----+------------------+
|12.0|11.585839692451506|
|8.69| 9.152692573629178|
| 6.3|6.3453607362976605|
|0.88|0.8427440476190476|
|1.97|2.1576583333333335|
+----+------------------+



### Model Evaluation

In [153]:
from pyspark.ml.evaluation import RegressionEvaluator

Evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='crew', metricName='r2')

In [154]:
r2_score = Evaluator.evaluate(predDF)

In [155]:
r2_score

0.995053709131534

In [136]:
# import math
# crew = predDF.select('crew')
# crew['LogCrew'] = crew['crew'].apply(math.log).hist(bins=300)

In [137]:
pipelineModel.write().overwrite().save('MyModel')

In [138]:
from pyspark.ml import  PipelineModel

saveModel = PipelineModel.load('MyModel')

In [139]:
saveModel

PipelineModel_71285da87069

By Eng. Mostafa Nabieh 
If you have questions, please feel free to ask.

My Email : nabieh.mostafa@yahoo.com

My Whatsapp : +201015197566